In [23]:
import findspark
import pyspark
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

In [24]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# Torna o pyspark "importável"
import findspark
findspark.init()

In [33]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Criar uma sessão Spark
spark = SparkSession.builder \
    .appName("Exemplo PySpark") \
    .getOrCreate()

# Verificar se a sessão foi criada corretamente
print(spark)

# Carregar o arquivo CSV em um DataFrame
arquivo = r"C:\Users\victoria\Documents\MeusProjetos- GIT\SoulCode\Bootcamp - ANALISE DE DADOS\Banco de Dados_base_suja.csv"
df_base_suja= spark\
        .read.format("csv")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .csv(arquivo)
df_base_suja.show()

+---+--------------+-----+----------+-------+-------------------+-----------------+
| ID|          Nome|Idade|      Área|Salário|Data de Contratação|Status de Emprego|
+---+--------------+-----+----------+-------+-------------------+-----------------+
|  1|    João Silva|   29|Financeiro|   5500|         01/02/2020|            Ativo|
|  2|   Maria Souza|   30|        RH|   4800|         15-03-2019|            ativo|
|  3|Carlos Pereira| NULL| Finaceiro|   6200|         2020/04/01|            Ativo|
|  4|     Ana Clara|   27| Marketing|   4800|         12/06/2018|            Ativo|
|  5|  Fabio Santos|   31|      NULL|   5500|         20-11-2017|            ativo|
|  6|   Sandra Lima|   28|        RH|   NULL|         05-05-2020|            Ativo|
|  7|    José Alves|   34| Marketing|   5400|         2018/02/01|          inativo|
|  8| Luciana Costa|   30|Financeiro|R$ 5200|         01.01.2019|            Ativo|
|  9| Paulo Ricardo| NULL| Finaceiro|   6100|         12/12/2020|          I

In [39]:
#1. Análise de Consistência de Dados: 
# Identificar e corrigir inconsistências nos valores de Área e Status de Emprego(e.g.,diferenças de maiúsculas/minúsculas).

# Renomear a coluna "Área" para "Area" e converter as colunas para minúsculas em uma única linha
status_area = df_base_suja.withColumnRenamed("Área", "Area") \
    .selectExpr("lower(Area) as Area", "lower(`Status de Emprego`) as `Status de Emprego`")

# Mostrar o resultado
status_area.show()

+----------+-----------------+
|      Area|Status de Emprego|
+----------+-----------------+
|financeiro|            ativo|
|        rh|            ativo|
| finaceiro|            ativo|
| marketing|            ativo|
|      NULL|            ativo|
|        rh|            ativo|
| marketing|          inativo|
|financeiro|            ativo|
| finaceiro|          inativo|
|        rh|            ativo|
|financeiro|            ativo|
+----------+-----------------+



In [ ]:
# 2. Limpeza de Dados Faltantes: Detectar e tratar os valores ausentes na coluna "Idade" e "Área".
